In [11]:
def recommend_with_svd(A,energy_percentage=0.9):
    import numpy as np
    import matplotlib.pyplot as plt
    
    # SVD
    m = A.shape[0]
    n = A.shape[1]
    U,s,VT = np.linalg.svd(A, full_matrices=False)
    
    s2 = s**2
    for i in range(1,len(s)):
        if np.sum(s2[:i]) / np.sum(s2) >= energy_percentage:
            k = i
            break
    
    A_reconstructed = np.dot(U[:,:k],np.dot(np.diag(s[:k]),VT[:k,:]))
    for i in range(m):
        for j in range(n):
            if A_reconstructed[i,j] == 0:
                for k in range(m):
                    if A_reconstructed[k,j] != 0:
                        cos_sim = cos_sim(A_reconstructed[i,:],A_reconstructed[k,:])
                        A_reconstructed[i,j] += cos_sim*A_reconstructed[k,j]
                        sum_sim += cos_sim
                A_reconstructed[i,j] /= sum_sim
    return A_reconstructed
            

In [12]:
def generate_evaluation_matrix(m,n):
    import numpy as np
    return np.random.randint(0,101,(m,n))

In [13]:
def cos_sim(a,b):
    import numpy as np
    return 0.5*np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))+0.5

In [14]:
A = generate_evaluation_matrix(10,10)
print(A)

[[ 49   7  85  20  29  95  43  79   7  46]
 [ 70  18  59 100   9  41  84  61   1  22]
 [  0  21  46  93  51  18  55  28  36  91]
 [ 76  72  63  24   5  30  59  19  94  25]
 [ 21   3  52  58  20  92  11   3  67  98]
 [ 87 100  38   6  79  24  50  67  45  28]
 [ 99  87  61  11  74  55  37  77  72  76]
 [ 12  16  58  69  11  72  10  52  76  31]
 [ 85  91  29  81  39  65  99  26  43  96]
 [ 29  35  14  23  34  89  93  10   6  51]]
